<a href="https://colab.research.google.com/github/vrajpatel2312/Crime-Analysis/blob/master/Clean_data_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

borough =['BRONX','BROOKLYN','STATEN ISLAND','QUEENS','MANHATTAN']

def gen_f(borough):
  rate = pd.read_csv('/content/drive/My Drive/datasets/generated.csv')
  months = pd.DataFrame(columns=['months'])
  month = [1,2,3,4,5,6,7,8,9,10,11,12]*3
  months['months']= month
  PATH='/content/drive/My Drive/datasets/'+borough+'.csv'
  data = pd.read_csv(PATH,index_col=0)
  data = pd.DataFrame(data,columns=['Year','Jan%'	,'Feb%'	,'Mar%'	,'Apr%'	,'May%'	,'Jun%'	,'Jul%'	,'Aug%'	,'Sep%'	,'Oct%'	,'Nov%'	,'Dec%'])
  data= data.drop(columns=['Year'])
  data = data.transpose().reset_index().rename(columns={'index':'Month'})
  data.columns.name = None
  for col in data.columns:
    data[col] = data[col].str.rstrip("%")
  data=pd.melt(data, id_vars =['Month'], value_vars =[2019, 2018,2017])

  data['Month']= month
  x_test =data[data['value'].isnull()]
  x_test = x_test.drop(columns=['value'])
  data = data.dropna()
  x_train= data.drop(columns=['value'])
  y_train = data.drop(columns=['Month','variable'])
  model = LinearRegression()
  model = model.fit(x_train,y_train)
  y_pred = model.predict(x_test)
  x_test['value']= y_pred
  output = data.dropna()
  output = x_test.append(output)
  rate=rate.drop(columns=['Unnamed: 0'])
  table = pd.pivot_table(rate, values='count', index=['month'],columns=['region','year'], aggfunc=np.sum, fill_value=0)
  null = ['NaN']*12
  rate= null+pd.Series.tolist(table[(borough,2018)])+pd.Series.tolist(table[(borough,2017)])
  output['rate']=rate
  PATH='/content/drive/My Drive/datasets/'+borough+'_cleaned.csv'
  output = output.to_csv(PATH)

for i in borough:
  gen_f(i)